In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

 ········


In [4]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader
dataset_name = "SaylorTwift/bbh"
page_content_column = "input"


loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

In [5]:
docs = loader.load()

In [6]:
docs[:5]

[Document(metadata={'target': '(C)'}, page_content='"Find a movie similar to Batman, The Mask, The Fugitive, Pretty Woman:\\nOptions:\\n(A) The Front Page\\n(B) Maelstrom\\n(C) The Lion King\\n(D) Lamerica"'),
 Document(metadata={'target': '(D)'}, page_content='"Find a movie similar to The Sixth Sense, The Matrix, Forrest Gump, The Shawshank Redemption:\\nOptions:\\n(A) Street Fighter II The Animated Movie\\n(B) The Sheltering Sky\\n(C) The Boy Who Could Fly\\n(D) Terminator 2 Judgment Day"'),
 Document(metadata={'target': '(C)'}, page_content='"Find a movie similar to Schindler\'s List, Braveheart, The Silence of the Lambs, Tombstone:\\nOptions:\\n(A) Orlando\\n(B) Guilty of Romance\\n(C) Forrest Gump\\n(D) All the Real Girls"'),
 Document(metadata={'target': '(E)'}, page_content='"Find a movie similar to Terminator 2 Judgment Day, The Fugitive, The Shawshank Redemption, Dead Man Walking:\\nOptions:\\n(A) Walk\\n(B) Don\'t Run\\n(C) Shaun the Sheep Movie\\n(D) Rocky IV\\n(E) Bravehear

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [9]:
os.environ["PINECONE_API_KEY"] = getpass.getpass()

 ········


In [10]:
from langchain_pinecone import PineconeVectorStore
index_name = "langchainstarter"

# Connect to Pinecone index and insert the chunked docs as contents
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'ea26a39115da099177d898fd5309b5eb', 'Date': 'Thu, 10 Oct 2024 17:19:52 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
retriever = docsearch.as_retriever()

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="API_KEY")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("Find a movie similar to to Batman")